# MATERIAS

In [52]:
# Cada clave es un semestre (columna). El orden sigue la foto.

semestres = {
    1: [
        "Pre Cálculo",
        "Cálculo 1",
        "Matemáticas Discretas",
        "Introducción a la Ciencia de Datos",
        "Programación 1",
        "Instituciones Políticas",
        "Pensamiento Crítico en Ciencia de Datos 1",
        "Idioma 1",
    ],
    2: [
        "Cálculo 2",
        "Álgebra Lineal",
        "Probabilidad",
        "Programación 2",
        "Ética en Ciencia de Datos",
        "Fundamento de Estadística",
        "Idioma 2",
    ],
    3: [
        "Inferencia Estadística",
        "Data Mining",
        "Estructuras de Datos",
        "Introducción a la Economía",
        "Seguridad y Privacidad",
        "Idioma 3",
    ],
    4: [
        "Estadística Bayesiana",
        "Machine Learning 1",
        "Bases de Datos",
        "Disciplina 1",
        "Pensamiento Crítico en Ciencia de Datos 2",
        "Idioma 4",
    ],
    5: [
        "Visualización 1",
        "Métodos Numéricos",
        "Machine Learning 2",
        "Big Data",
        "Disciplina 2",
        "Emprendimiento",
        "Idioma 5",
    ],
    6: [
        "Visualización 2",
        "Optimización",
        "Deep Learning",
        "Formulación de Proyectos",
        "Disciplina 3",
        "Idioma 6",
    ],
    7: [
        "Prácticas",
    ],
    8: [
        "Taller de Habilidades Profesionales",
        "Ciberseguridad",
        "Proyectos",
        "Disciplina 4",
        "Taller de Habilidades Gerenciales",
    ],
}



# CLASE PROYECCIÓN

In [ ]:
import math
import pandas as pd

class Proyector:
    # Materias con tope 20 por grupo
    MATERIAS_MAX20 = {
        "Pre Cálculo","Cálculo 1", "Cálculo 2", "Matemáticas Discretas", "Álgebra Lineal",
        "Probabilidad", "Programación 1", "Programación 2", "Inferencia Estadística",
        "Data Mining", "Estructuras de Datos", "Estadística Bayesiana",
        "Machine Learning 1", "Machine Learning 2", "Bases de Datos",
        "Métodos Numéricos", "Big Data", "Optimización", "Deep Learning"
    }
    # Constructor
    def __init__(self, redondeo: str | None = None):
        self.redondeo = redondeo
        self._rows: list[pd.DataFrame] = []

    #  Convierte un porcentaje a decimal
    def _pct_to_decimal(self, p):
        p = float(p)
        if p < 0: p = 0.0
        if p > 1: p = p / 100.0    # 24 -> 0.24
        return p

    # Aplica el esquema de redondeo
    def _rnd(self, x):
        if self.redondeo == "round":
            return int(round(float(x)))
        if self.redondeo == "ceil":
            return int(math.ceil(float(x)))
        return float(x)

    # Calcula la proyección para una materia
    def calcular(self, materia: str, porcentaje_perdida, inscritos: float) -> pd.DataFrame:
        p = self._pct_to_decimal(porcentaje_perdida)
        perdieron = inscritos * p
        proyectados = inscritos + perdieron   # (sobrevivientes: inscritos - perdieron)
        fila = {
            "Materia": materia,
            "Inscritos": self._rnd(inscritos),
            "Perdida_%": round(p * 100, 2),
            "Estudiantes_perdieron": self._rnd(perdieron),
            "Estudiantes_proyectados": self._rnd(proyectados),
        }
        return pd.DataFrame([fila])
    

    # Agrega una materia a la tabla
    def add(self, materia: str, porcentaje_perdida, inscritos: float):
        self._rows.append(self.calcular(materia, porcentaje_perdida, inscritos))
        return self
    

    # Devuelve la tabla completa
    def tabla(self) -> pd.DataFrame:
        if not self._rows:
            return pd.DataFrame(columns=[
                "Materia","Inscritos","Perdida_%","Estudiantes_perdieron","Estudiantes_proyectados"
            ])
        return pd.concat(self._rows, ignore_index=True)
    
    # Resetea la tabla
    def reset(self):
        self._rows.clear()
        return self

    # Divide las materias en grupos
    def dividir_grupos(self, df: pd.DataFrame | None = None, cap_otras: int = 35) -> pd.DataFrame:
        """
        Usa la tabla de este Proyector y añade:
        - Capacidad_usada (20 si la materia está restringida, si no cap_otras)
        - No_grupos
        - Tamaños_grupos (lista con el tamaño de cada grupo, repartidos parejo)

        Regla: si sobra 1 (o pocos), se redistribuye para evitar grupos de 1.
        """
        # 1) usa la tabla interna acumulada por la clase.
        base = self.tabla() if df is None else df.copy()

        # 2) Verifica que existan las columnas mínimas.
        if not {"Materia", "Estudiantes_proyectados"}.issubset(base.columns):
            raise ValueError("Se necesita un DataFrame con columnas 'Materia' y 'Estudiantes_proyectados'.")

        filas = []
        for _, row in base.iterrows():
            materia = str(row["Materia"])
            n = int(round(float(row["Estudiantes_proyectados"] or 0)))

            # 3) Elegir capacidad por grupo:
            cap = 20 if materia in self.MATERIAS_MAX20 else int(cap_otras)

            if n <= 0:
                grupos = 0
                sizes = []
            else:
                # 4) Número de grupos = techo(n/capacidad) - calcula cuántos grupos necesitas para que ninguno exceda la capacidad cap
                grupos = math.ceil(n / cap)
                # 5) Reparto equilibrado (evita grupos de 1)
                base_size = n // grupos # tamaño base parejo para cada grupo usando división entera
                extra = n % grupos # cuántos estudiantes sobran tras asignar base_size a cada grupo.
                sizes = [base_size + 1] * extra + [base_size] * (grupos - extra) # Construye la lista de tamaños por grupo, repartiendo el sobrante

            filas.append({
                "Materia": materia,
                "Estudiantes_proyectados": n,
                "Capacidad_usada": cap,
                "No_grupos": grupos,
                "Tamaños_grupos": sizes
            })

        out = pd.DataFrame(filas, columns=[
            "Materia", "Estudiantes_proyectados", "Capacidad_usada", "No_grupos", "Tamaños_grupos"
        ])
        # 6) Merge para conservar las columnas originales (Inscritos, Perdida_%, etc.)
        return base.merge(out, on=["Materia", "Estudiantes_proyectados"], how="left")



In [50]:
# Ejemplo 
proy = Proyector(redondeo="ceil")
proy.reset()\
    .add( "Matemáticas Discretas", 0.238095238, 82)\
    .add("Cálculo 1", 0.238095238, 19)\
    .add("Programación 1", 20, 84)\
    .add("Formulación de Proyectos", 0, 48)\
    .add("Big Data", 0, 41)\
    .add("Ética", 0, 61)\
    .add("Optimización", 0.15, 60)

tabla = proy.tabla()

grupos = proy.dividir_grupos(tabla)
display(grupos)


,Materia,Inscritos,Perdida_%,Estudiantes_perdieron,Estudiantes_proyectados,Capacidad_usada,No_grupos,Tamaños_grupos
0,Matemáticas Discretas,82,23.81,20,102,20,6,"[17, 17, 17, 17, 17, 17]"
1,Cálculo 1,19,23.81,5,24,20,2,"[12, 12]"
2,Programación 1,84,20.00,17,101,20,6,"[17, 17, 17, 17, 17, 16]"
3,Formulación de Proyectos,48,0.00,0,48,35,2,"[24, 24]"
4,Big Data,41,0.00,0,41,20,3,"[14, 14, 13]"
5,Ética,61,0.00,0,61,35,2,"[31, 30]"
6,Optimización,60,15.00,9,69,20,4,"[18, 17, 17, 17]"
